# FlavorFinder

Aplikacja pozwala przeglądać dostępne dania z darmowego api [mealAPI](https://www.themealdb.com), oraz dodać ulubione do lokalnej bazy danych.

Aplikacja wykorzystuje bazę danych `ROOM` do przechowywania ulubionych dań, oraz bibliotekę `Retrofit` do pobrania danych z zewnętrznego serwera.

<table><tr><td><img src="https://media0.giphy.com/media/v1.Y2lkPTc5MGI3NjExZzlweXR6dzI1b3ZpNnk4MXo3cTJ6Yjd1NWx3ZG85ZGNzNGkyOHZobCZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/NqBQ91k7zOVh5d39NK/giphy.gif" width="200" /></td><td><img src="https://media3.giphy.com/media/v1.Y2lkPTc5MGI3NjExY29iNXd2Z2hiYnhldWtpeHVpeTR5YmYwM3ltZWh3bHlyYmkwcTNneCZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/WiFtop3Zg3tdTUGFnh/giphy.gif" width="200" /></td><td><img src="https://media1.giphy.com/media/v1.Y2lkPTc5MGI3NjExOGs3ZTZmMnpsb21pdTAwbWphbGo1MmxoZGFtNmUwaGlzdmttcndwcCZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/Van1cMPDSfS9haqhIL/giphy.gif" width="200" /></td></tr></table>

- Aplikacja zaimplementowana zgodnie ze wzoccem MVVM wraz z repozytorium
- Repozytorium zawiera metody dostępowe do lokalnej bazy oraz zewnętrznego api
- biblioteka `Retrofit` została wykorzystana do połączenia z zewnętrznym serwerem
- biblioteka `Coil` została wykorzystana do obsługi ładowania grafik
- `LiveData` został wykorzystany do łatwego zarządzania bieżącym stanem interfejsu użytkownika
- Baza danych `Room` została wykorzystana do przechowywania dań
- Aplikacja zawiera trzy ekrany 
    - na głównym ekranie wyświetla listę dań pobranych z serwera
    - ekran *ulubione* pozwala przeglądać dania zapisane w lokalnej bazy danych
    - trzeci ekran umożliwia uzyskanie większej ilości informacji o wybranym daniu
- Aplikacja wykorzystuje `Jetpack Navigation` w celu utworzenia nawigacji w aplikacji
- `ViewModel` został umieszczony w aktywności głównej, w celu uzyskania współdzielenia jego instancji przez wszystkie ekrany


Dodajmy wymagane zależności do projektu

```kotlin
buildscript {
    repositories {
        google()
    }
    dependencies {
        classpath ("androidx.navigation:navigation-safe-args-gradle-plugin:2.5.1")
    }
}
plugins {
    id("com.android.application") version "8.1.1" apply false
}

-----
    
plugins {
    id("com.android.application")
    id ("androidx.navigation.safeargs")
}

dependencies {

    implementation ("com.squareup.retrofit2:retrofit:2.9.0")
    implementation ("com.google.code.gson:gson:2.9.1")
    implementation ("com.squareup.retrofit2:converter-gson:2.9.0")

    // ROOM
    implementation ("androidx.room:room-runtime:2.5.2")
    implementation("androidx.navigation:navigation-fragment:2.5.3")
    implementation("androidx.navigation:navigation-ui:2.5.3")
    annotationProcessor ("androidx.room:room-compiler:2.5.2")

    // ViewModel
    implementation ("androidx.lifecycle:lifecycle-viewmodel:2.5.1")
    // LiveData
    implementation ("androidx.lifecycle:lifecycle-livedata:2.5.1")

    implementation("io.coil-kt:coil:2.4.0")
    ...
}
```

dodajmy również odpowiednie upoważnienia do manifestu

```xml
<uses-permission android:name="android.permission.INTERNET"/>
```

Przejrzyjmy dane które wykorzystamy z api. W tej aplikacji pobierzemy tylko polskie dania, korzystając z endpointu https://www.themealdb.com/api/json/v1/1/filter.php?a=Polish. Zobaczmy odpowiedź.

In [ ]:
{
   "meals":[
      {
         "strMeal":"Bigos (Hunters Stew)",
         "strMealThumb":"https:\/\/www.themealdb.com\/images\/media\/meals\/md8w601593348504.jpg",
         "idMeal":"53018"
      },
      {
         "strMeal":"Go\u0142\u0105bki (cabbage roll)",
         "strMealThumb":"https:\/\/www.themealdb.com\/images\/media\/meals\/q8sp3j1593349686.jpg",
         "idMeal":"53021"
      },
      {
         "strMeal":"Paszteciki (Polish Pasties)",
         "strMealThumb":"https:\/\/www.themealdb.com\/images\/media\/meals\/c9a3l31593261890.jpg",
         "idMeal":"53017"
      },
      {
         "strMeal":"Pierogi (Polish Dumplings)",
         "strMealThumb":"https:\/\/www.themealdb.com\/images\/media\/meals\/45xxr21593348847.jpg",
         "idMeal":"53019"
      },
      {
         "strMeal":"Polskie Nale\u015bniki (Polish Pancakes)",
         "strMealThumb":"https:\/\/www.themealdb.com\/images\/media\/meals\/58bkyo1593350017.jpg",
         "idMeal":"53022"
      },
      {
         "strMeal":"Rogaliki (Polish Croissant Cookies)",
         "strMealThumb":"https:\/\/www.themealdb.com\/images\/media\/meals\/7mxnzz1593350801.jpg",
         "idMeal":"53024"
      },
      {
         "strMeal":"Ros\u00f3\u0142 (Polish Chicken Soup)",
         "strMealThumb":"https:\/\/www.themealdb.com\/images\/media\/meals\/lx1kkj1593349302.jpg",
         "idMeal":"53020"
      },
      {
         "strMeal":"Sledz w Oleju (Polish Herrings)",
         "strMealThumb":"https:\/\/www.themealdb.com\/images\/media\/meals\/7ttta31593350374.jpg",
         "idMeal":"53023"
      }
   ]
}

Dostaniemy nazwę, link do zdjęcia oraz identyfikator. Chcemy również dostać więcej informacji o wybranym przez użytkownika daniu, w tym celu wykorzystamy endpoint https://www.themealdb.com/api/json/v1/1/lookup.php?i=52772. Po podaniu identyfikatora otrzymamy następujące dane.

In [ ]:
{
   "meals":[
      {
         "idMeal":"52772",
         "strMeal":"Teriyaki Chicken Casserole",
         "strDrinkAlternate":null,
         "strCategory":"Chicken",
         "strArea":"Japanese",
         "strInstructions":"Preheat oven to 350\u00b0 F. Spray a 9x13-inch baking pan with non-stick spray.\r\nCombine soy sauce, ...   
         "strMealThumb":"https:\/\/www.themealdb.com\/images\/media\/meals\/wvpsxx1468256321.jpg",
         "strTags":"Meat,Casserole",
         "strYoutube":"https:\/\/www.youtube.com\/watch?v=4aZr5hZXP_s",
         "strIngredient1":"soy sauce",
         "strIngredient2":"water",
         "strIngredient3":"brown sugar",
         "strIngredient4":"ground ginger",
         "strIngredient5":"minced garlic",
         "strIngredient6":"cornstarch",
         "strIngredient7":"chicken breasts",
         "strIngredient8":"stir-fry vegetables",
         "strIngredient9":"brown rice",
         "strIngredient10":"",
         "strIngredient11":"",
         "strIngredient12":"",
         "strIngredient13":"",
         "strIngredient14":"",
         "strIngredient15":"",
         "strIngredient16":null,
         "strIngredient17":null,
         "strIngredient18":null,
         "strIngredient19":null,
         "strIngredient20":null,
         "strMeasure1":"3\/4 cup",
         "strMeasure2":"1\/2 cup",
         "strMeasure3":"1\/4 cup",
         "strMeasure4":"1\/2 teaspoon",
         "strMeasure5":"1\/2 teaspoon",
         "strMeasure6":"4 Tablespoons",
         "strMeasure7":"2",
         "strMeasure8":"1 (12 oz.)",
         "strMeasure9":"3 cups",
         "strMeasure10":"",
         "strMeasure11":"",
         "strMeasure12":"",
         "strMeasure13":"",
         "strMeasure14":"",
         "strMeasure15":"",
         "strMeasure16":null,
         "strMeasure17":null,
         "strMeasure18":null,
         "strMeasure19":null,
         "strMeasure20":null,
         "strSource":null,
         "strImageSource":null,
         "strCreativeCommonsConfirmed":null,
         "dateModified":null
      }
   ]
}

## Data

Wybierzmy tylko pola, które będą nas interesować i stwórzmy model. Dane dostaniemy w obu przypadkach jako listę, więc dodajmy klasę `MealRepsonse` zawierającą listę obiektów typu `Meal`. W klasie `Meal` definiujemy tylko pola, które nas interesują. Dla obu odpowiedzi możemy posłużyć się jednym modelem (nie ma konieczności tworzenia wielu modeli, jeżeli przynajmniej część danych się powtarza)

In [ ]:
public class MealResponse {
    public ArrayList<Meal> meals;
}

In [ ]:
@Entity(tableName = "meals")
public class Meal {
    @PrimaryKey
    @NonNull
    public String idMeal;
    public String strMeal;
    public String strCategory;
    public String strArea;
    public String strInstructions;
    public String strMealThumb;
    public String strTags;
    public String strYoutube;
}


Dodajmy interfejs reprezentujący api, i zdefiniujmy w nim dwie metody, które zwracają wszystkie dania, oraz danie o zadanym id.

In [ ]:
public interface MealApi {
    @GET("api/json/v1/1/filter.php?a=Polish")
    Call<MealResponse> getMealFromApi();

    @GET("api/json/v1/1/lookup.php?")
    Call<MealResponse> getMealById(@Query("i")String id);
}

Następnie dodajmy obiekt reprezentujący instancję `Retrofit`

In [ ]:
public class RetrofitInstance {
    private RetrofitInstance(){}

    private static volatile MealApi api;
    private static final String baseUrl = "https://www.themealdb.com/";

    public static MealApi getApi() {
        if (api == null) {
            synchronized (RetrofitInstance.class) {
                if (api == null) {
                    api = new Retrofit.Builder()
                            .baseUrl(baseUrl)
                            .addConverterFactory(GsonConverterFactory.create())
                            .build().create(MealApi.class);
                }
            }
        }
        return api;
    }
}

Dodajmy `Dao` dla bazy `ROOM` ze zdefiniowanymi metodami dla dodawania, usuwania, oraz zwrócenia wszystkich elementów lokalnej bazy

In [ ]:
@Dao
public interface MealDao {
    @Insert(onConflict = REPLACE)
    void insert(Meal meal);

    @Delete
    void delete(Meal meal);

    @Query("SELECT * FROM meals")
    LiveData<List<Meal>> getAllMeals();
}


Następnie dodajmy klasę abstrakcyjną reprezentującą naszą bazę danych.

In [ ]:
@Database(entities = {Meal.class}, version = 1, exportSchema = false)
public abstract class MealDatabase extends RoomDatabase {
    public abstract MealDao mealDao();

    private static volatile MealDatabase INSTANCE;

    public static MealDatabase getDatabase(final Context context) {
        if (INSTANCE == null) {
            synchronized (MealDatabase.class) {
                if (INSTANCE == null) {
                    INSTANCE = Room.databaseBuilder(context.getApplicationContext(),
                                    MealDatabase.class, "meal_database")
                            .build();
                }
            }
        }
        return INSTANCE;
    }
}


Dodajmy repozytorium z metodami dostępowymi.

In [ ]:
public class MealRepository {

    private final MealDao mealDao;

    public MealRepository(Application application) {
        MealDatabase db = MealDatabase.getDatabase(application);
        mealDao = db.mealDao();
    }

    public Call<MealResponse> getMealFromApi(){
        return RetrofitInstance.getApi().getMealFromApi();
    }

    public Call<MealResponse> getMealById(String id){
        return RetrofitInstance.getApi().getMealById(id);
    }

    public void insert(Meal meal){
        mealDao.insert(meal);
    }

    public void delete(Meal meal){
        mealDao.delete(meal);
    }

    public LiveData<List<Meal>> getAllMeals(){
        return mealDao.getAllMeals();
    }
}

## ViewModel

Wykorzystamy również wzorzec `ResourceBound` w celu ułatwienia reakcji na różne stany.

In [ ]:
public enum Status {
    SUCCESS,
    ERROR,
    LOADING
}

In [ ]:
public class Resource<T> {
    @NonNull
    public final Status status;
    @Nullable
    public final T data;
    @Nullable public final String message;

    private Resource(@NonNull Status status, @Nullable T data, @Nullable String message) {
        this.status = status;
        this.data = data;
        this.message = message;
    }

    public static <T> Resource<T> success(@NonNull T data) {
        return new Resource<>(SUCCESS, data, null);
    }

    public static <T> Resource<T> error(String msg, @Nullable T data) {
        return new Resource<>(ERROR, data, msg);
    }

    public static <T> Resource<T> loading(@Nullable T data) {
        return new Resource<>(LOADING, data, null);
    }
}

Z bazą będziemy komunikować się asynchronicznie, więc dodajmy klasę pomocniczą dla obsługi wielowątkowości

In [ ]:
public class AppExecutors {
    private static final Object LOCK = new Object();
    private static AppExecutors instance;
    private final Executor dbIO;

    private AppExecutors(Executor dbIO) {
        this.dbIO = dbIO;
    }

    public static AppExecutors getInstance() {
        if (instance == null) {
            synchronized (LOCK) {
                instance = new AppExecutors(Executors.newSingleThreadExecutor());
            }
        }
        return instance;
    }

    public Executor diskIO() {
        return dbIO;
    }
}

Rozpocznijmy implementację `ViewModel` od dodania pól

In [ ]:
public class MealViewModel extends AndroidViewModel {

    private final String TAG = "MealViewModel";

    private final MealRepository repository;

    private final MutableLiveData<Resource<MealResponse>> meals = new MutableLiveData<>();
    private final MutableLiveData<Resource<MealResponse>> meal = new MutableLiveData<>();
    private LiveData<List<Meal>> favorites;

    public MealViewModel(@NonNull Application application) {
        super(application);
        repository = new MealRepository(application);
    }

    public LiveData<Resource<MealResponse>> getMeals() {
        return meals;
    }

    public LiveData<Resource<MealResponse>> getMeal() {
        return meal;
    }

    public LiveData<List<Meal>> getFavorites() {
        return favorites;
    }
    ...
}

- `meals` przechowuje listę wszystkich dań pobranych z serwera, inicjujemy stanem ładowania
- `meal` przechowuje informację o wybranym daniu pobraną z serwera, inicjujemy stanem ładowania
- `favorites` przechowuje listę wszystkich dań dodanych do listy ulubionych i zapisanych w lokalnej bazie danych

Dodajmy metodę pomocniczą do obsługi odpowiedzi z serwera.

W kolejnym kroku zaimplementujmy metody `getMealsFromApi` oraz `getMealById` pobierające dane z serwera.

In [ ]:
    public void getMealById(String id){
        Call<MealResponse> responseCall = repository.getMealById(id);

        responseCall.enqueue(new Callback<MealResponse>() {
            @Override
            public void onResponse(@NonNull Call<MealResponse> call, @NonNull Response<MealResponse> response) {
                if (response.isSuccessful()){
                    meal.postValue(Resource.loading(null)); // inicjujemy stan ładowania
                    MealResponse mealResponse = response.body();
                    if (mealResponse != null)
                        meal.postValue(Resource.success(mealResponse)); // w przypadku sukcesu, zmieniamy stan i przekazujemy dane
                }
            }

            @Override
            public void onFailure(@NonNull Call<MealResponse> call, @NonNull Throwable t) {
                meal.postValue(Resource.error(t.getLocalizedMessage(), null)); // w przypadku błędu ustawiamy odpowiedni stan
            }
        });
    }

    public void getMealsFromApi() {
        Call<MealResponse> responseCall = repository.getMealFromApi();

        responseCall.enqueue(new Callback<MealResponse>() {
            @Override
            public void onResponse(@NonNull Call<MealResponse> call, @NonNull Response<MealResponse> response) {
                meals.postValue(Resource.loading(null));
                if (response.isSuccessful()){
                    MealResponse mealsResponse = response.body();
                    if (mealsResponse != null)
                        meals.postValue(Resource.success(mealsResponse));
                }
            }

            @Override
            public void onFailure(@NonNull Call<MealResponse> call, @NonNull Throwable t) {
                meals.postValue(Resource.error(t.getLocalizedMessage(), null));
            }
        });
    }

Wywołujemy odpowiednią funkcję z repozytorium. Ostanim krokiem w tej funkcji jest wywołanie `handleMealResponse`, która zwraca odpowiedni stan (sukces z danymi, lub błąd z odpowiednią wiadomością).

Dodajmy dwie metody obsługujące dodanie i usunięcie elementu z lokalnej bazy.

In [ ]:
    public void insert(Meal meal){
        AppExecutors.getInstance().diskIO().execute(() -> repository.insert(meal));
    }

    public void delete(Meal meal){
        AppExecutors.getInstance().diskIO().execute(() -> repository.delete(meal));
    }

Pełny kod `ViewModel`

In [ ]:
public class MealViewModel extends AndroidViewModel {

    private final String TAG = "MealViewModel";

    private final MealRepository repository;

    private final MutableLiveData<Resource<MealResponse>> meals = new MutableLiveData<>();
    private final MutableLiveData<Resource<MealResponse>> meal = new MutableLiveData<>();
    private LiveData<List<Meal>> favorites;

    public MealViewModel(@NonNull Application application) {
        super(application);
        repository = new MealRepository(application);
    }

    public LiveData<Resource<MealResponse>> getMeals() {
        return meals;
    }

    public LiveData<Resource<MealResponse>> getMeal() {
        return meal;
    }

    public LiveData<List<Meal>> getFavorites() {
        return favorites;
    }

    public void getMealById(String id){
        Call<MealResponse> responseCall = repository.getMealById(id);

        responseCall.enqueue(new Callback<MealResponse>() {
            @Override
            public void onResponse(@NonNull Call<MealResponse> call, @NonNull Response<MealResponse> response) {
                if (response.isSuccessful()){
                    meal.postValue(Resource.loading(null));
                    MealResponse mealResponse = response.body();
                    if (mealResponse != null)
                        meal.postValue(Resource.success(mealResponse));
                }
            }

            @Override
            public void onFailure(@NonNull Call<MealResponse> call, @NonNull Throwable t) {
                meal.postValue(Resource.error(t.getLocalizedMessage(), null));
            }
        });
    }

    public void getMealsFromApi() {
        Call<MealResponse> responseCall = repository.getMealFromApi();

        responseCall.enqueue(new Callback<MealResponse>() {
            @Override
            public void onResponse(@NonNull Call<MealResponse> call, @NonNull Response<MealResponse> response) {
                meals.postValue(Resource.loading(null));
                if (response.isSuccessful()){
                    MealResponse mealsResponse = response.body();
                    if (mealsResponse != null)
                        meals.postValue(Resource.success(mealsResponse));
                }
            }

            @Override
            public void onFailure(@NonNull Call<MealResponse> call, @NonNull Throwable t) {
                meals.postValue(Resource.error(t.getLocalizedMessage(), null));
            }
        });
    }

    public void getLocalMeals(){
        favorites = repository.getAllMeals();
    }

    public void insert(Meal meal){
        AppExecutors.getInstance().diskIO().execute(() -> repository.insert(meal));
    }

    public void delete(Meal meal){
        AppExecutors.getInstance().diskIO().execute(() -> repository.delete(meal));
    }
}

## Interfejs użytkownika

Wykorzystamy `BottomNavigation`, na którym umieścimy dwa ekrany - wyświetlający listę pobraną z serwera, oraz listę pobraną z lokalnej bazy. Dodamy również trzeci ekran (poza `BottomNavigation`) wyświetlający widok szczegółowy.

Dodajmy nawigację

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<navigation xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    xmlns:tools="http://schemas.android.com/tools"
    android:id="@+id/navigation"
    app:startDestination="@id/foodListFragment">
    <fragment
        android:id="@+id/foodListFragment"
        android:name="com.example.flavorfinderjava.ui.fragments.FoodListFragment"
        android:label="fragment_food_list"
        tools:layout="@layout/fragment_food_list" >
        <action
            android:id="@+id/action_foodListFragment_to_detailFragment"
            app:destination="@id/detailFragment" />
    </fragment>
    <fragment
        android:id="@+id/favoriteFragment"
        android:name="com.example.flavorfinderjava.ui.fragments.FavoriteFragment"
        android:label="fragment_favorite"
        tools:layout="@layout/fragment_favorite" >
        <action
            android:id="@+id/action_favoriteFragment_to_detailFragment"
            app:destination="@id/detailFragment" />
    </fragment>
    <fragment
        android:id="@+id/detailFragment"
        android:name="com.example.flavorfinderjava.ui.fragments.DetailFragment"
        android:label="fragment_detail"
        tools:layout="@layout/fragment_detail" >
        <action
            android:id="@+id/action_detailFragment_to_foodListFragment"
            app:destination="@id/foodListFragment" />
        <action
            android:id="@+id/action_detailFragment_to_favoriteFragment"
            app:destination="@id/favoriteFragment" />
    </fragment>
</navigation>

Następnie dodajmy menu dla `BottomNavigation`

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<menu xmlns:android="http://schemas.android.com/apk/res/android">
    <item
        android:id="@id/foodListFragment"
        android:icon="@drawable/ic_food"
        android:title="Home" />
    <item
        android:id="@id/favoriteFragment"
        android:icon="@drawable/ic_favorite_border"
        android:title="Favorite" />
</menu>

Dodajmy layout dla głównej aktywności.

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<LinearLayout xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:tools="http://schemas.android.com/tools"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    android:layout_width="match_parent"
    android:layout_height="match_parent"
    android:orientation="vertical">

    <androidx.fragment.app.FragmentContainerView
        android:id="@+id/nav_host_fragment"
        android:name="androidx.navigation.fragment.NavHostFragment"
        android:layout_width="match_parent"
        android:layout_height="0dp"
        android:layout_weight="1"
        app:defaultNavHost="true"
        app:navGraph="@navigation/navigation"/>

    <com.google.android.material.bottomnavigation.BottomNavigationView
        android:id="@+id/bottom_nav_view"
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        app:menu="@menu/bottom_navigation_menu" />

</LinearLayout>

Połączmy nawigację w aktywności głównej

In [ ]:
public class MainActivity extends AppCompatActivity {
    private NavController navController;

    @Override
    protected void onCreate(Bundle savedInstanceState) {
        super.onCreate(savedInstanceState);
        ActivityMainBinding binding = ActivityMainBinding.inflate(getLayoutInflater());
        setContentView(binding.getRoot());

        NavHostFragment navHostFragment = (NavHostFragment) getSupportFragmentManager()
                .findFragmentById(R.id.nav_host_fragment);

        if (navHostFragment != null) {
            navController = NavHostFragment.findNavController(navHostFragment);
            NavigationUI.setupWithNavController(binding.bottomNavView, navController);
        }

    }
}

Przejdźmy do layoutów fragmentów.

W layoucie przeznaczonym do wyświetlania listy pobranej z api dodamy `ProgressBar`, który będzie pokazywany w stanie `Resource.Loading`

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<androidx.constraintlayout.widget.ConstraintLayout xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    xmlns:tools="http://schemas.android.com/tools"
    android:layout_width="match_parent"
    android:layout_height="match_parent">

    <androidx.recyclerview.widget.RecyclerView
        android:id="@+id/foodRV"
        android:layout_width="match_parent"
        android:layout_height="0dp"
        android:layout_margin="8dp"
        app:layout_constraintEnd_toEndOf="parent"
        app:layout_constraintStart_toStartOf="parent"
        app:layout_constraintTop_toTopOf="parent" />

    <ProgressBar
        android:id="@+id/progressBar"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:layout_gravity="center"
        android:background="@android:color/transparent"
        android:visibility="invisible"
        app:layout_constraintBottom_toBottomOf="parent"
        app:layout_constraintEnd_toEndOf="parent"
        app:layout_constraintStart_toStartOf="parent"
        app:layout_constraintTop_toTopOf="parent" />

</androidx.constraintlayout.widget.ConstraintLayout>

W layoucie przeznaczonym do wyświetlania listy pobranej z lokalnej bazy dodamy tylko `RecyclerView`

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<androidx.constraintlayout.widget.ConstraintLayout xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    xmlns:tools="http://schemas.android.com/tools"
    android:layout_width="match_parent"
    android:layout_height="match_parent">

    <androidx.recyclerview.widget.RecyclerView
        android:id="@+id/favoriteRV"
        android:layout_width="match_parent"
        android:layout_height="0dp"
        android:layout_margin="8dp"
        app:layout_constraintEnd_toEndOf="parent"
        app:layout_constraintStart_toStartOf="parent"
        app:layout_constraintTop_toTopOf="parent" />

</androidx.constraintlayout.widget.ConstraintLayout>

Layout wyświetlający widok szczegółowy jest nieco bardziej skopmplikowany i również zawiera `ProgressBar`. Ponadto dodamy `FAB` dzięki któremu użytkownik będzie miał możliwość dodania elementu do bazy lokalnej. Usunięcie zrealizujemy przez dodanie funkcjonalności `SwipeToDelete`. Opis elementu może nie zmieścić się na ekranie, więc całość zagnieżdżamy w `ScrollView`

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<ScrollView xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:tools="http://schemas.android.com/tools"
    android:layout_width="match_parent"
    android:layout_height="match_parent">

    <RelativeLayout
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        tools:context=".DetailActivity">

        <ProgressBar
            android:id="@+id/progressBar"
            android:layout_width="wrap_content"
            android:layout_height="wrap_content"
            android:layout_alignParentStart="true"
            android:layout_alignParentTop="true"
            android:layout_alignParentEnd="true"
            android:layout_gravity="center"
            android:layout_marginStart="181dp"
            android:layout_marginTop="35dp"
            android:layout_marginEnd="182dp"
            android:background="@android:color/transparent"
            android:visibility="invisible" />

        <ImageView
            android:id="@+id/foodImage"
            android:layout_width="match_parent"
            android:layout_height="wrap_content"
            android:adjustViewBounds="true"
            android:contentDescription="" />

        <TextView
            android:id="@+id/title"
            android:layout_width="wrap_content"
            android:layout_height="wrap_content"
            android:layout_alignBottom="@id/foodImage"
            android:padding="8dp"
            android:text=""
            android:theme="@style/ThemeOverlay.AppCompat.Dark" />

        <TextView
            android:id="@+id/category"
            android:layout_width="wrap_content"
            android:layout_height="wrap_content"
            android:layout_below="@id/foodImage"
            android:padding="8dp"
            android:text=""
            android:textColor="?android:textColorSecondary" />

        <TextView
            android:id="@+id/instructions"
            android:layout_width="wrap_content"
            android:layout_height="wrap_content"
            android:layout_below="@id/category"
            android:padding="8dp"
            android:text="" />

        <com.google.android.material.floatingactionbutton.FloatingActionButton
            android:id="@+id/favoriteButton"
            android:layout_width="wrap_content"
            android:layout_height="wrap_content"
            android:layout_alignBottom="@id/foodImage"
            android:layout_alignParentEnd="true"
            android:layout_marginEnd="8dp"
            android:layout_marginTop="16dp"
            android:contentDescription=""
            android:src="@drawable/ic_favorite_border" />
    </RelativeLayout>
</ScrollView>

`ProgressBar` wymaga zdefiniowania layoutu dla samego *spinnera*

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<TextView xmlns:android="http://schemas.android.com/apk/res/android"
    android:id="@+id/spinner_layout"
    android:layout_width="match_parent"
    android:layout_height="wrap_content"
    android:ellipsize="marquee"
    android:fontFamily="sans-serif"
    android:gravity="center"
    android:singleLine="true"
    android:text=""
    android:padding="10dp"
    android:textSize="24sp" />

Ostatnim layoutem jest element listy `RecyclerView`, skorzystamy z jednego pliku dla obu wykorzystanych list.

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<LinearLayout xmlns:android="http://schemas.android.com/apk/res/android"
    android:layout_width="match_parent"
    android:layout_height="wrap_content"
    android:orientation="vertical"
    android:layout_marginTop="16dp">

    <TextView
        android:id="@+id/name"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:layout_gravity="center"
        android:text="nazwa"
        android:textSize="24sp"
        android:layout_margin="8dp"/>

    <ImageView
        android:id="@+id/image"
        android:layout_width="match_parent"
        android:layout_gravity="center"
        android:layout_height="200dp"
        android:scaleType="centerCrop"
        android:layout_marginStart="25dp"
        android:layout_marginEnd="25dp"
        android:contentDescription="" />

</LinearLayout>

Ponieważ korzystamy z `RecyclerView` musimy dodać klasy wspomagające. W tej aplikacji korzystamy z dwóch list, jednak zaimplementujemy tylko jeden adapter. Funkcję `OnClick` elementu listy przekażemy jako *lambdę*. W tym celu musimy dodać interfejs funkcyjny.

In [ ]:
public interface OnItemClickListener {
    void onItemClick(String mealId);
}

In [ ]:
public class MealComparator extends DiffUtil.ItemCallback<Meal> {
    @Override
    public boolean areItemsTheSame(@NonNull Meal oldItem, @NonNull Meal newItem) {
        return newItem.idMeal.equals(oldItem.idMeal);
    }

    @Override
    public boolean areContentsTheSame(@NonNull Meal oldItem, @NonNull Meal newItem) {
        return newItem.strMeal.equals(oldItem.strMeal);
    }
}

In [ ]:
public class MealViewHolder extends RecyclerView.ViewHolder {
    private final ListItemRvBinding binding;
    private final OnItemClickListener onItemClickListener;
    public MealViewHolder(ListItemRvBinding binding, OnItemClickListener onItemClickListener) {
        super(binding.getRoot());
        this.binding = binding;
        this.onItemClickListener = onItemClickListener;
    }

    public void bind(Meal item){
        binding.name.setText(item.strMeal);

        Context context = binding.getRoot().getContext();
        ImageLoader imageLoader = Coil.imageLoader(context);
        ImageRequest request = new ImageRequest.Builder(context)
                .data(item.strMealThumb)
                .target(binding.image)
                .build();
        imageLoader.enqueue(request);

        binding.getRoot().setOnClickListener(v -> onItemClickListener.onItemClick(item.idMeal));
    }
}


In [ ]:
public class MealAdapter extends ListAdapter<Meal, MealViewHolder> {
    private final OnItemClickListener onItemClickListener;

    public MealAdapter(OnItemClickListener onItemClickListener, MealComparator comparator) {
        super(comparator);
        this.onItemClickListener = onItemClickListener;
    }

    @NonNull
    @Override
    public MealViewHolder onCreateViewHolder(@NonNull ViewGroup parent, int viewType) {
        return new MealViewHolder(ListItemRvBinding.inflate(
                LayoutInflater.from(parent.getContext()), parent, false),
                onItemClickListener);
    }

    @Override
    public void onBindViewHolder(@NonNull MealViewHolder holder, int position) {
        Meal item = getItem(position);
        holder.bind(item);
    }

    public Meal getItemAt(int position){
        return getItem(position);
    }
}


Dodajmy fragmenty

In [ ]:
public class FoodListFragment extends Fragment { // wyświetla listę pobraną z api

    private FragmentFoodListBinding binding;
    private MealViewModel viewModel;

    @Override
    public View onCreateView(@NonNull LayoutInflater inflater, ViewGroup container,
                             Bundle savedInstanceState) {
        binding = FragmentFoodListBinding.inflate(inflater, container, false);

        viewModel = new ViewModelProvider(requireActivity()).get(MealViewModel.class);
        viewModel.getMealsFromApi();

        return binding.getRoot();
    }

    @Override
    public void onViewCreated(@NonNull View view, @Nullable Bundle savedInstanceState) {
        super.onViewCreated(view, savedInstanceState);

        MealAdapter adapter = new MealAdapter(
                this::onItemClick,
                new MealComparator());

        setupRecyclerView(adapter);

        viewModel.getMeals().observe(getViewLifecycleOwner(), mealResponse -> { // oberwator danych
            if (mealResponse.status == Status.SUCCESS) {
                binding.progressBar.setVisibility(View.INVISIBLE); // przestawiam widoczność ProgressBar w przypadku sukcesu
                if (mealResponse.data != null) {
                    List<Meal> data = mealResponse.data.meals;
                        adapter.submitList(data); // ładuję dane do adaptera
                    }
                } else if (mealResponse.status == Status.ERROR) { // jeśeli jest błąd
                    binding.progressBar.setVisibility(View.INVISIBLE); // ukrywam ProgressBar
                    Toast.makeText(requireContext(), mealResponse.message, Toast.LENGTH_SHORT).show(); // pokazuję informację o błędzie
                } else if (mealResponse.status == Status.LOADING) { // stan ładowania
                    binding.progressBar.setVisibility(View.VISIBLE); // pokazuję ProgressBar
                }
        });
    }

    private void onItemClick(String mealId) { // funkcja przekazana jako parametr do adaptera - OnClick elementu listy
        viewModel.getMealById(mealId); // wywołuję funkcję, unikając konieczności przekazania id
        NavDirections action = FoodListFragmentDirections // nawigacja do DetailFragment
            .actionFoodListFragmentToDetailFragment();
        Navigation.findNavController(binding.getRoot()).navigate(action);
    }

    private void setupRecyclerView(MealAdapter adapter){
        binding.foodRV.setAdapter(adapter);
        binding.foodRV.setLayoutManager(new LinearLayoutManager(requireContext()));
    }
}

In [ ]:
public class FavoriteFragment extends Fragment { // wyświetla dane z ROOM

    private FragmentFavoriteBinding binding;
    private MealViewModel viewModel;

    @Override
    public View onCreateView(@NonNull LayoutInflater inflater, ViewGroup container,
                             Bundle savedInstanceState) {
        binding = FragmentFavoriteBinding.inflate(inflater, container, false);
        viewModel = new ViewModelProvider(requireActivity()).get(MealViewModel.class);
        viewModel.getLocalMeals();
        return binding.getRoot();
    }

    @Override
    public void onViewCreated(@NonNull View view, @Nullable Bundle savedInstanceState) {
        super.onViewCreated(view, savedInstanceState);

        MealAdapter adapter = new MealAdapter(
                this::onItemClick,
            new MealComparator()
        );
        setupRecyclerView(adapter);

        viewModel.getFavorites().observe(getViewLifecycleOwner(), adapter::submitList);

        swipeToDelete(adapter);
    }

    private void setupRecyclerView(MealAdapter adapter){
        binding.favoriteRV.setAdapter(adapter);
        binding.favoriteRV.setLayoutManager(new LinearLayoutManager(requireContext()));
    }

    private void swipeToDelete(MealAdapter adapter) { // usunięcie elementu przez przeciągnięcie w lewo lub prawo
        new ItemTouchHelper(new ItemTouchHelper.SimpleCallback(0,
                ItemTouchHelper.RIGHT | ItemTouchHelper.LEFT) {
            @Override
            public boolean onMove(@NonNull RecyclerView recyclerView, @NonNull RecyclerView.ViewHolder viewHolder, @NonNull RecyclerView.ViewHolder target) {
                return false;
            }

            @Override
            public void onSwiped(@NonNull RecyclerView.ViewHolder viewHolder, int direction) {
                viewModel.delete(adapter.getItemAt(viewHolder.getAdapterPosition()));
            }
        }).attachToRecyclerView(binding.favoriteRV);
    }

    private void onItemClick(String mealId) { // funkcja przekazana jako parametr do adaptera - OnClick elementu listy
        viewModel.getMealById(mealId); // wywołuję funkcję, unikając konieczności przekazania id
        NavDirections action = FavoriteFragmentDirections // nawigacja do DetailFragment
                .actionFavoriteFragmentToDetailFragment();
        Navigation.findNavController(binding.getRoot()).navigate(action);
    }
}

In [ ]:
public class DetailFragment extends Fragment { // ładuje dane szczegółowe z api
    private FragmentDetailBinding binding;
    private MealViewModel viewModel;

    @Override
    public View onCreateView(@NonNull LayoutInflater inflater, ViewGroup container,
                             Bundle savedInstanceState) {
        binding = FragmentDetailBinding.inflate(inflater, container, false);

        viewModel = new ViewModelProvider(requireActivity()).get(MealViewModel.class);

        return binding.getRoot();
    }

    @Override
    public void onViewCreated(@NonNull View view, @Nullable Bundle savedInstanceState) {
        super.onViewCreated(view, savedInstanceState);

        viewModel.getMeal().observe(getViewLifecycleOwner(), mealResponse -> {
            if (mealResponse.data != null){
                Optional<Meal> mealOptional = mealResponse // odpakowanie danych
                        .data
                        .meals
                        .stream()
                        .findFirst();

                mealOptional.ifPresent(this::inflate); // jeżeli dane istnieją, wywołuję funkcję inflate

                binding.favoriteButton.setOnClickListener(v -> viewModel.insert(
                        mealOptional.orElse(null) // dodanie elementu do lokalnej bazy
                ));
            }
        });
    }

    private void inflate(Meal item){ // funkcja pomocnicza ładujące dane do layoutu

        Context context = requireContext();
        ImageLoader imageLoader = Coil.imageLoader(context);
        ImageRequest request = new ImageRequest.Builder(context)
                .data(item.strMealThumb)
                .target(binding.foodImage)
                .build();
        imageLoader.enqueue(request);

        binding.category.setText(item.strCategory);
        binding.title.setText(item.strMeal);
        binding.instructions.setText(item.strInstructions);
    }
}

Możemy przetestować aplikację.

<img src="https://media0.giphy.com/media/v1.Y2lkPTc5MGI3NjExZzlweXR6dzI1b3ZpNnk4MXo3cTJ6Yjd1NWx3ZG85ZGNzNGkyOHZobCZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/NqBQ91k7zOVh5d39NK/giphy.gif" width="200" />